In [9]:
!pip install spotipy

In [10]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

In [11]:
CLIENT_ID = "d3b16b119e9547d6965785618bcf4543"
CLIENT_SECRET = "30e320c3376a4383ba5a25af9bc8cb2a"
REDIRECT_URI = "http://localhost:8080"
SCOPE = "playlist-read-private"
PLAYLIST_IDS = ["2LEWfVqIUsbBcwLzX1GzKs"]  # Replace or add more playlist IDs
all_data = []

# --- Authenticate with OAuth ---
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE
))



In [12]:
for pid in PLAYLIST_IDS:
    playlist = sp.playlist(pid)
    playlist_name = playlist['name']
    print(f"📥 Fetching: {playlist_name}")
    
    tracks = sp.playlist_items(pid, limit=100)["items"]

    for item in tracks:
        track = item['track']
        if not track:
            continue

        # Basic track data
        song = track["name"]
        artist_name = track["artists"][0]["name"]
        artist_id = track["artists"][0]["id"]
        popularity = track["popularity"]
        explicit = track["explicit"]
        duration_ms = track["duration_ms"]
        album_name = track["album"]["name"]
        release_date = track["album"]["release_date"]

        # Fetch artist data
        genres = []
        artist_popularity = None
        artist_followers = None

        if artist_id:
            try:
                artist_info = sp.artist(artist_id)
                genres = artist_info.get("genres", [])
                artist_popularity = artist_info.get("popularity")
                artist_followers = artist_info.get("followers", {}).get("total")
            except Exception as e:
                print(f"⚠️ Error fetching artist data for {artist_name}: {e}")

        all_data.append({
            "Playlist": playlist_name,
            "Song": song,
            "Artist": artist_name,
            "Popularity": popularity,
            "Explicit": explicit,
            "Duration (ms)": duration_ms,
            "Album": album_name,
            "Release Date": release_date,
            "Artist Popularity": artist_popularity,
            "Artist Followers": artist_followers,
            "Genres": ", ".join(genres) if genres else "unknown"
        })

# --- Save to CSV ---
df = pd.DataFrame(all_data)
df.to_csv("2024_playlist_with_artist_album_info.csv", index=False)
print("✅ Saved to playlist_with_artist_album_info.csv")


📥 Fetching: 2024
✅ Saved to playlist_with_artist_album_info.csv
